### Dixon Coles Model
https://penaltyblog.readthedocs.io/en/latest/ratings/massey_ratings.html

https://www.pro-football-reference.com/years/2022/games.htm


In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
import os
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import penaltyblog as pb

In [3]:
import nfl_data_py as nfl

In [4]:
try:
    import nfl_data_py as nfl
except:
    ! pip install nfl_data_py

In [5]:
semana = 8
temporada = 2022
normalizacion = 1
MAX = 10
df = nfl.import_schedules([temporada])
df = df[df['week']<semana]

In [6]:
df = df.rename(columns={'away_team':'AwayTeam',
                                   'home_team':'HomeTeam',
                       'away_score':'FTHG',
                       'home_score':'FTAG'})


In [7]:
df[['FTHG','FTAG']]

,FTHG,FTAG
6137,31.0,10.0
6138,27.0,26.0
6139,26.0,24.0
6140,10.0,19.0
6141,23.0,20.0
...,...,...
6240,20.0,38.0
6241,37.0,23.0
6242,44.0,23.0
6243,10.0,16.0


In [8]:
df[df['AwayTeam']=='BUF']

,game_id,season,game_type,week,gameday,weekday,gametime,AwayTeam,FTHG,HomeTeam,...,wind,away_qb_id,home_qb_id,away_qb_name,home_qb_name,away_coach,home_coach,referee,stadium_id,stadium
6137,2022_01_BUF_LA,2022,REG,1,2022-09-08,Thursday,20:20,BUF,31.0,LA,...,NaN,00-0034857,00-0026498,Josh Allen,Matthew Stafford,Sean McDermott,Sean McVay,Carl Cheffers,LAX01,SoFi Stadium
6173,2022_03_BUF_MIA,2022,REG,3,2022-09-25,Sunday,13:00,BUF,19.0,MIA,...,NaN,00-0034857,00-0036212,Josh Allen,Tua Tagovailoa,Sean McDermott,Mike McDaniel,Alex Kemp,MIA00,Hard Rock Stadium
6188,2022_04_BUF_BAL,2022,REG,4,2022-10-02,Sunday,13:00,BUF,23.0,BAL,...,NaN,00-0034857,00-0034796,Josh Allen,Lamar Jackson,Sean McDermott,John Harbaugh,Jerome Boger,BAL00,M&T Bank Stadium
6228,2022_06_BUF_KC,2022,REG,6,2022-10-16,Sunday,16:25,BUF,24.0,KC,...,NaN,00-0034857,00-0033873,Josh Allen,Patrick Mahomes,Sean McDermott,Andy Reid,Brad Allen,KAN00,GEHA Field at Arrowhead Stadium


In [9]:
if normalizacion == 1:
    df['FTHG'] = (df['FTHG'] - df['FTHG'].min()) / ( df['FTHG'].max() - df['FTHG'].min()) * MAX
    df['FTAG'] = (df['FTAG'] - df['FTAG'].min()) / ( df['FTAG'].max() - df['FTAG'].min()) * MAX

In [10]:
df[["FTHG", "FTAG"]].mean()

FTHG    4.390432
FTAG    4.603175
dtype: float64

In [11]:
df[['FTHG','FTAG']]

,FTHG,FTAG
6137,6.458333,1.666667
6138,5.625000,5.476190
6139,5.416667,5.000000
6140,2.083333,3.809524
6141,4.791667,4.047619
...,...,...
6240,4.166667,8.333333
6241,7.708333,4.761905
6242,9.166667,4.761905
6243,2.083333,3.095238


In [12]:
pr = nfl.import_schedules([temporada])
pr = pr[pr['week']==semana ]
juegos = pr['week'].count()

In [13]:
pr = pr.rename(columns={'away_team':'AwayTeam',
                                   'home_team':'HomeTeam',
                       'away_score':'FTHG',
                       'home_score':'FTAG'})

In [14]:
model = pb.models.DixonColesGoalModel(df["FTHG"], df["FTAG"], df["HomeTeam"], df["AwayTeam"])
model.fit()

In [15]:
print('Dixon Coles Model')
print(f'Season: {temporada}')
print(f'Week: {semana}')
for i in range(0,juegos):
    EL = pr.iloc[i]['HomeTeam']
    EV = pr.iloc[i]['AwayTeam']
    probs = model.predict(EL, EV)
    if probs.home_win >= .50 :
        print(f'L {EL} -{100/probs.home_win:.0f} vs V {EV} {100/probs.away_win:.0f}')
    elif probs.away_win >= .50:
        print(f'L {EL} {100/probs.home_win:.0f} vs V {EV} -{100/probs.away_win:.0f}')
    else:
        print(f'L {EL} {100/probs.home_win:.0f} vs V {EV} {100/probs.away_win:.0f}')

Dixon Coles Model
Season: 2022
Week: 8
L TB -156 vs V BAL 449
L JAX 393 vs V DEN -176
L ATL 271 vs V CAR 207
L DAL 599 vs V CHI -145
L DET -159 vs V MIA 385
L MIN 310 vs V ARI -190
L NO -183 vs V LV 296
L NYJ 290 vs V NE -200
L PHI 1018 vs V PIT -122
L HOU -159 vs V TEN 408
L IND 261 vs V WAS 226
L LA -183 vs V SF 334
L SEA -154 vs V NYG 429
L BUF 2591 vs V GB -110
L CLE -141 vs V CIN 538


In [16]:
model

Module: Penaltyblog

Model: Dixon and Coles

Number of parameters: 66
Log Likelihood: -418.574
AIC: 969.148

Team                 Attack               Defence             
------------------------------------------------------------
ARI                  1.059                0.295               
ATL                  1.141                0.381               
BAL                  0.968                0.708               
BUF                  0.331                0.686               
CAR                  1.005                0.138               
CHI                  1.014                0.12                
CIN                  0.784                0.509               
CLE                  1.249                0.495               
DAL                  0.537                0.277               
DEN                  0.709                -0.271              
DET                  1.495                0.692               
GB                   1.096                0.244               
HOU        

In [17]:
model.get_params()

{'attack_ARI': 1.059078931946171,
 'attack_ATL': 1.1407618558909134,
 'attack_BAL': 0.9677917188436387,
 'attack_BUF': 0.3307953682087275,
 'attack_CAR': 1.0047497276017514,
 'attack_CHI': 1.0137698065322718,
 'attack_CIN': 0.7841339565240943,
 'attack_CLE': 1.24882221384117,
 'attack_DAL': 0.5373937180091068,
 'attack_DEN': 0.7094053066717987,
 'attack_DET': 1.4953478399485585,
 'attack_GB': 1.0963751666021715,
 'attack_HOU': 1.23380557364547,
 'attack_IND': 0.931448775341382,
 'attack_JAX': 0.9875041149961116,
 'attack_KC': 1.2322301707408976,
 'attack_LA': 1.028614785611095,
 'attack_LAC': 1.1770927039269032,
 'attack_LV': 1.2477898161589196,
 'attack_MIA': 1.0664032985415324,
 'attack_MIN': 0.8575126377041253,
 'attack_NE': 0.8681795766621923,
 'attack_NO': 1.4076363781197712,
 'attack_NYG': 0.8943200981808658,
 'attack_NYJ': 0.8953731647590104,
 'attack_PHI': 0.7956562350524856,
 'attack_PIT': 1.074641161375747,
 'attack_SEA': 1.2782173408317217,
 'attack_SF': 0.8107777763624335,


In [29]:
#Listado de Librerías instaladas
import subprocess

sub = subprocess.run(["pip", "list", '--disable-pip-version-check'],
                    stdout=subprocess.PIPE,
                    universal_newlines=True
                    )
pkgs = (line.rstrip().split() for line in sub.stdout.split("\n")[2: -1])

for paquete, version in pkgs:
    print(paquete, version)

absl-py 1.2.0
aeppl 0.0.33
aesara 2.7.9
appdirs 1.4.4
appnope 0.1.2
argon2-cffi 21.3.0
argon2-cffi-bindings 21.2.0
arviz 0.12.1
asttokens 2.0.5
astunparse 1.6.3
async-generator 1.10
attrs 21.4.0
backcall 0.2.0
beautifulsoup4 4.11.1
bidict 0.22.0
bleach 4.1.0
cachetools 5.2.0
certifi 2022.6.15
cffi 1.15.1
cftime 1.6.1
charset-normalizer 2.1.0
cloudpickle 2.1.0
configparser 5.3.0
cons 0.4.5
cramjam 2.5.0
cssselect 1.1.0
cycler 0.11.0
debugpy 1.5.1
decorator 5.1.1
defusedxml 0.7.1
deprecat 2.1.1
dill 0.3.5.1
empiricaldist 0.6.7
entrypoints 0.4
etuples 0.3.5
executing 0.8.3
facenet-pytorch 2.5.2
fastjsonschema 2.15.1
fastparquet 0.8.3
fastprogress 1.0.3
filelock 3.8.0
flatbuffers 1.12
fonttools 4.34.4
fsspec 2022.8.2
fxcmpy 1.3.3
gast 0.4.0
google-auth 2.10.0
google-auth-oauthlib 0.4.6
google-pasta 0.2.0
graphviz 0.20.1
grpcio 1.47.0
h11 0.13.0
h5py 3.7.0
idna 3.3
imageio 2.21.1
importlib-metadata 4.12.0
ipykernel 6.9.1
ipython 8.4.0
ipython-genutils 0.2.0
jedi 0.18.1
Jinja2 3.0.3
joblib 1